# Funciones para encontrar usuarios a testear

In [13]:
import pandas as pd
import requests
from IPython.display import HTML, display
from bs4 import BeautifulSoup
from bs4 import Comment
import json
import html
import re
import urllib.parse
from threading import Thread
from tqdm import tqdm
import pickle

In [2]:
#configuro proxy y headers
url_proxy = 'https://free-proxy-list.net/anonymous-proxy.html'
response_pxy = requests.get(url_proxy)
pxySoup = BeautifulSoup(response_pxy.text, 'html.parser')
rowsPxy = []
headPxy = []
for tablePart in pxySoup.find_all(attrs={'id':'proxylisttable'})[0].find_all(True, recursive=False):
    if(tablePart.name == 'thead'):
        for row in tablePart.find_all(True, recursive=False):
            headPxy = [tr.text for tr in row]
    if(tablePart.name == 'tbody'):
        for row in tablePart.find_all(True, recursive=False):
            rowsPxy.append([tr.text for tr in row])
proxiesDf = pd.DataFrame(data=rowsPxy, columns=headPxy)
proxiesDf = proxiesDf[(proxiesDf['Anonymity']=='elite proxy')].sort_values(by='Https', ascending=False)
headersChrome = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
}
proxies = {
    "http": proxiesDf[proxiesDf.Https=='no'].loc[:,'IP Address'].values[0]
}

In [10]:
# targetUsers = pd.DataFrame(columns=['username'])

targetUsers = pd.read_pickle('targetUsersDataset.pkl')
print(len(targetUsers))
targetUsers.head()

9080


,username
235491971,gsrcaballero
497428913,juannfalopa
1536923558,FabianPereyra87
2300560809,NMoraniB
179471208,Sergioogz


In [5]:
headersMore = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Host': 'twitter.com',
    'Accept-Language': 'es-AR,es;q=0.8,en-US;q=0.5,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'X-Twitter-Active-User': 'yes',
    'X-Requested-With': 'XMLHttpRequest',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

def getNewPage(term, searchType, maxPosition):
    if(searchType != 'q'):
        term = '#'+term
    pageResponse = requests.get(
        'https://twitter.com/i/search/timeline',
        params={
            'vertical': 'news',
            'q': term,
            'max_position': maxPosition,
            'include_entities': 1,
            'include_available_features': 1,
            'reset_error_state': False
        },
        proxies=proxies,
        headers=headersMore
    )
    return(pageResponse.json())
#     print(pageResponse.json().keys())
#     print(pageResponse.json()['min_position'])
#     display(BeautifulSoup(pageResponse.json()['items_html'], 'html.parser'))
    return pageResponse.json()['min_position'], BeautifulSoup(pageResponse.json()['items_html'], 'html.parser')

def getUserTweetData(tweet):
    try:
        username = tweet.findAll(attrs={'class':'username'})[0].text[1:]
        userId = tweet.findAll('div', attrs={'class':'js-stream-tweet'})[0].attrs['data-user-id']
        targetUsers.loc[userId] = [username]
    except:
        print('ERROR---------------------')
        print(tweet)
    return [username, userId]

def searchTerm(term, searchType='q', maxPages=15):
    pageResponse = False
    if(searchType == 'q'):
        pageResponse = requests.get('https://twitter.com/search?q='+term, headers=headersChrome, proxies=proxies)
    else:
        pageResponse = requests.get('https://twitter.com/hashtag/'+term, headers=headersChrome, proxies=proxies)
    twData = BeautifulSoup(pageResponse.text, 'html.parser')
    initialStream = twData.findAll(attrs={'id':'timeline'})[0].findAll(attrs={'class':'stream-container'})[0]
    dataMax = initialStream.attrs['data-max-position']
    dataMin = initialStream.attrs['data-min-position']
    streamItems = initialStream.findAll(attrs={'id':'stream-items-id'})[0]
    tweetList = streamItems.findAll('li', attrs={'data-item-type': "tweet"}, recursive=False)
    for tweet in tweetList:
        getUserTweetData(tweet)
    nextKey = dataMax
    for pageNum in tqdm(range(maxPages)):
        response = getNewPage(term, searchType, nextKey)
        nextKey = response['min_position']
        htmlResponse = BeautifulSoup(response['items_html'], 'html.parser')
        if(len(htmlResponse.findAll('li', attrs={'data-item-type': "tweet"}, recursive=False)) == 0):
            print(htmlResponse)
            break
        for tweet in htmlResponse.findAll('li', attrs={'data-item-type': "tweet"}, recursive=False):
            getUserTweetData(tweet)

# searchTerm('cfk', '#', 5)
# searchTerm('cambiemos', '#', 5)
# searchTerm('MacriVendepatria', '#', 5)
# searchTerm('JuntosPorElCambio', '#', 5)
# searchTerm('DefensoresDelCambio', '#', 5)

In [14]:
def process_id(id):
    """process a single ID"""
    # fetch the data
    r = requests.get(url_t % id)
    # parse the JSON reply
    data = r.json()
    # and update some data with PUT
    requests.put(url_t % id, data=data)
    return data

def process_range(id_range, store=None):
    """process a number of ids, storing the results in a dict"""
    if store is None:
        store = {}
    for id in id_range:
        store[id] = searchTerm(id, 'q', 600)
    return store

def threaded_process_range(nthreads, id_range):
    """process the id range in a specified number of threads"""
    store = {}
    threads = []
    # create the threads
    for i in range(nthreads):
        ids = id_range[i::nthreads]
        t = Thread(target=process_range, args=(ids,store))
        threads.append(t)

    # start the threads
    [ t.start() for t in threads ]
    # wait for the threads to finish
    [ t.join() for t in threads ]
    print('Done')
    print(len(targetUsers))
    display(targetUsers.head())
    return 'Done'

termsToSearch = [
    'cfk',
    'MacriVendepatria',
    'yovotoalfrentedetodos',
    'ElijoALosFernandez',
    'FernandezFernandez2019',
    'macriespobreza',
    'macrigato',
    'FueraMacri',
    
    'cambiemos',
    'JuntosPorElCambio',
    'DefensoresDelCambio',
    'SigamosCambiandoJuntos',
    'ListaCompletaDelCambio',
    'MacriPichetto2019',
    'NoVuelvenMas',
    'EllaMiente'
]
termsToSearch = [
    'macri',
    'vidal',
    'macri gato',
    'macri es pobreza'
]
termsToSearch = [
    'macri voto',
    'vidal voto',
    'cistina voto',
    'fernandez voto'
]

termsToSearch = [
    'macri inflacion',
    'vidal inflacion',
    'cistina inflacion',
    'fernandez inflacion',
    'elecciones 2019',
]
threaded_process_range(8, termsToSearch)
filename = 'targetUsersDataset.pkl'
with open(filename, 'wb') as twUsersTarget:
    pickle.dump(targetUsers, twUsersTarget)
    
termsToSearch = [
    'macri FMI',
    'sinceramente falso',
    'ella miente',
    'massa',
    'cristina pobreza'
]
threaded_process_range(8, termsToSearch)
filename = 'targetUsersDataset.pkl'
with open(filename, 'wb') as twUsersTarget:
    pickle.dump(targetUsers, twUsersTarget)







  0%|                                                                                          | 0/600 [00:00<?, ?it/s]






  0%|                                                                                          | 0/600 [00:00<?, ?it/s]







  0%|                                                                                          | 0/600 [00:00<?, ?it/s]








  0%|                                                                                          | 0/600 [00:00<?, ?it/s]









  0%|                                                                                          | 0/600 [00:00<?, ?it/s]














  0%|▏                                                                                 | 1/600 [00:01<10:10,  1.02s/it]





  0%|▏                                                                                 | 1/600 [00:01<13:45,  1.38s/it]







  0%|▏                                                                                 | 1/600 [00:01<16:13,  1.62s/it]








  0%|▏                                                                                 | 1/600 [00:01<17:22,  1.74s/it]







  0%|▎                                                                                 | 2/600 [00:02<13:33,  1.36s/it]







  0%|▎                                                                                 | 2/600 [00:03<15:54,  1.60s/it]








  0%|▎                                                                                 | 2/600 [00:03<16:42,  1.68s/it]





  0%|▍                                                                                 | 3/600 [00:03<13:04,  1.31s/it]







  0%|▍                                                                                 | 3/600 [00:04<14:58,  1.51s/it]








  0%|▍                                                                                 | 3/600 [00:04<16:00,  1.61s/it]





  1%|▌                                                                                 | 4/600 [00:05<13:53,  1.40s/it]







  1%|▌                                                                                 | 4/600 [00:06<15:16









  2%|█▏                                                                                | 9/600 [00:13<14:33,  1.48s/it]





  2%|█▎                                                                               | 10/600 [00:14<13:51,  1.41s/it]







  2%|█▎                                                                               | 10/600 [00:15<14:40,  1.49s/it]





  2%|█▍                                                                               | 11/600 [00:15<14:04,  1.43s/it]







  2%|█▍                                                                               | 11/600 [00:16<14:30,  1.48s/it]





  2%|█▌                                                                               | 12/600 [00:16<13:51,  1.41s/it]







  2%|█▌                                                                               | 12/600 [00:18<15:13,  1.55s/it]





  2%|█▊                                                                               | 13/600 [00:18<14:14,  1

  7%|█████▋                                                                           | 42/600 [01:01<14:39,  1.58s/it]







  7%|█████▋                                                                           | 42/600 [01:02<13:49,  1.49s/it]





  7%|█████▊                                                                           | 43/600 [01:03<14:10,  1.53s/it]







  7%|█████▊                                                                           | 43/600 [01:03<13:27,  1.45s/it]





  7%|█████▉                                                                           | 44/600 [01:04<14:09,  1.53s/it]







  7%|█████▉                                                                           | 44/600 [01:05<13:25,  1.45s/it]







  8%|██████                                                                           | 45/600 [01:06<13:13,  1.43s/it]







  8%|██████▏                                                                          | 46/600 [01:08<13:00,  1.41s

...











  5%|████▏                                                                            | 31/600 [00:48<14:59,  1.58s/it]










  5%|████▎                                                                            | 32/600 [00:48<15:14,  1.61s/it]









  5%|████▎                                                                            | 32/600 [00:50<14:24,  1.52s/it]










  6%|████▍                                                                            | 33/600 [00:49<14:33,  1.54s/it]









  6%|████▍                                                                            | 33/600 [00:51<14:51,  1.57s/it]










  6%|████▌                                                                            | 34/600 [00:51<14:58,  1.59s/it]









  6%|████▌                                                                            | 34/600 [00:53<14:22,  1.52s/it]










  6%|████▋                                                                         

 10%|████████▎                                                                        | 62/600 [01:34<12:56,  1.44s/it]










 11%|████████▋                                                                        | 64/600 [01:34<12:53,  1.44s/it]









 10%|████████▌                                                                        | 63/600 [01:35<12:41,  1.42s/it]










 11%|████████▊                                                                        | 65/600 [01:35<12:38,  1.42s/it]









 11%|████████▋                                                                        | 64/600 [01:36<12:19,  1.38s/it]










 11%|████████▉                                                                        | 66/600 [01:36<12:35,  1.42s/it]









 11%|████████▊                                                                        | 65/600 [01:38<12:27,  1.40s/it]










 11%|█████████                                                                        | 67/60











 15%|████████████▍                                                                    | 92/600 [02:16<10:35,  1.25s/it]









 16%|████████████▌                                                                    | 93/600 [02:17<10:34,  1.25s/it]









 16%|████████████▋                                                                    | 94/600 [02:18<10:32,  1.25s/it]









 16%|████████████▊                                                                    | 95/600 [02:19<10:25,  1.24s/it]









 16%|████████████▉                                                                    | 96/600 [02:21<10:33,  1.26s/it]









 16%|█████████████                                                                    | 97/600 [02:22<11:05,  1.32s/it]









 16%|█████████████▏                                                                   | 98/600 [02:24<10:57,  1.31s/it]









 16%|█████████████▎                                                                   |



Done
15805


,username
235491971,gsrcaballero
497428913,juannfalopa
1536923558,FabianPereyra87
2300560809,NMoraniB
179471208,Sergioogz


In [12]:
#guardar resultados
import pickle
filename = 'targetUsersDataset.pkl'
with open(filename, 'wb') as twUsersTarget:
    pickle.dump(targetUsers, twUsersTarget)